In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pandas.plotting import autocorrelation_plot
from plotting_functions import \
run_diagnostic_plots, plot_acf, plot_trace, plot_sensor_output, get_fname_list, plot_layer_posteriors, plot_fieldobs, plot_hist_diagnostic, make_chainstat_table
import visvis as vv
import diagplots2
import pymc3
import matplotlib

import geovis_notebook_version

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_csv('/Users/davidkohn/Desktop/last_stats_table.csv',sep='\s+')

In [ ]:
df_subsection_list = [range(0,12), range(12,24), range(24, 36), range(36, 48)]

In [ ]:
df_list = [df.loc[l,:] for l in df_subsection_list]

In [ ]:
df_list[3].shape

In [ ]:
df

In [ ]:
data = np.abs(df_list[1].diff(axis = 0))['Beta'].values

In [ ]:
data = [df_list_i['Beta'].values for df_list_i in df_list]

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.gca()
x = list(range(1,13))
for y in data:
    plt.plot(x, np.log(y))
    ax.set_xticks(x)

In [ ]:
df_list[0]

In [ ]:
df_list[0].loc[1:10,'Beta']

In [ ]:
num = df_list[0].loc[1:11,'Beta'].values
denom = df_list[0].loc[0:10,'Beta'].values
data = denom/num

In [ ]:
data

In [ ]:
df_list[0].loc[:,'Beta']

In [ ]:
plt.plot(data)

In [ ]:
df_list[0].loc[1:11,'Beta']/df_list[0].loc[0:10,'Beta']

In [ ]:
df_list[2]

In [ ]:
author = 'dk' # 'dk' or 'rs

rp_param_title_list = ['Rock density', 'Magnetic susceptibility']
rp_layer_title_list = ['Layer 0', 'Layer 1']

if author == 'dk': # user input - for david's results
    dir_parent = '/Volumes/david_hd/obsidian/output/experiments/'
    dir_output_diagnostics = 'output/plots-diagnostic'
    dir_output_sensors = 'output/plots-sensors'
    fname_samples = 'output.npz'

    dir_year = '2018'
    dir_month = '11'
    dir_day = '10'
    exp_str = '{}_{}_{}'.format(dir_month, dir_day, dir_year)
    num_list = ['01']

    param_key_list = [
        'layer0rockProperties',
        'layer1rockProperties'
    ]
    param_idx_list = [0, 1]

    plot_function_list = [
        plot_acf,
        plot_trace,
        plot_hist_diagnostic,
        make_chainstat_table
    ]

    plot_function_names = [
        'acf',
        'trace',
        'hist',
        'chainstat-table',
    ]
    extension_list = [
        'eps',
        'eps',
        'eps',
        'csv',
    ]
    # for sensor plot output
    sensor_name_list = ['grav', 'mag'] #'grav' or 'mag'

    data_names_list = [
        'magSensors','magReadings','gravSensors','gravReadings'
    ]
    xp = lambda exp_str, num: os.path.join(exp_str, num)

elif author == 'rs': # user input - for richard's results
    dir_parent = '/Volumes/david_hd/obsidian/output/experiments/'
    dir_output_diagnostics = 'output/plots-diagnostic'
    dir_output_sensors = 'output/plots-sensors'
    #fname_samples = 'gascoyne_v4.npz'

    exp_str = 'gascoyne_v5_run'
    fname_samples = 'gascoyne_v5-rs-run03-thin1000.npz'
    #fname_samples = 'output.npz'
    #num_list = ['01', '02', '03', '04', '05', '06']
    num_list = ['03']

    param_key_list = [
        'layer0rockProperties',
        'layer1rockProperties'
    ]
    param_idx_list = [0, 1]

    plot_function_list = [
        plot_acf,
        plot_trace,
        plot_hist_diagnostic,
        make_chainstat_table
    ]

    plot_function_names = [
        'acf',
        'trace',
        'hist',
        'chainstat-table'
    ]
    extension_list = [
        'eps',
        'eps',
        'eps',
        'csv',
    ]
    # for sensor plot output
    sensor_name_list = ['grav', 'mag'] #'grav' or 'mag'

    data_names_list = [
        'magSensors','magReadings','gravSensors','gravReadings'
    ]
    xp = lambda exp_str, num: exp_str + num

# Diagnostic plots

In [ ]:
# put together file paths
if author == 'dk':
    fpath_in_list = [
        os.path.join(dir_parent, exp_str, num, fname_samples)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str, num, dir_output_diagnostics)
        for num in num_list
    ]
elif author == 'rs':
    fpath_in_list = [
        os.path.join(dir_parent, exp_str + num, fname_samples)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str + num, dir_output_diagnostics)
        for num in num_list
    ]
print(fpath_in_list)
print(dir_out_list)

In [ ]:
layer_title_list = ['Layer 0', 'Layer 1']
param_title_list = ['Rock density', 'Magnetic susceptibility']

In [ ]:
# make plots
stack_length_list = [1160161, 1154009, 1129264, 1111255]
run_diagnostic_plots(
    fpath_in_list,
    dir_out_list,
    param_key_list,
    param_idx_list,
    plot_function_list,
    extension_list,
    plot_function_names,
    layer_title_list,
    param_title_list,
    n_burn = 0,
    n_thin = 1000,
    stack_length_list = stack_length_list
)

In [ ]:
"""# testing with single plot
run_diagnostic_plots(
    fpath_in_list,
    dir_out_list,
    [param_key_list[0]],
    [param_idx_list[0]],
    [plot_function_list[1]],
    [plot_function_names[1]],
    [layer_title_list[0]],
    [param_title_list[0]],
    burn_in = 100
)"""

# Sensor plots

In [ ]:
# put together file paths
#subsample_csv = False
#fname_samples = 'output.npz'
#fname_samples = 'gascoyne_v5-rs-run02-thin1000.npz'

data_names_list_sensorplots = data_names_list
#data_names_list_sensorplots = ['magSensors_subsamp', 'magReadings_subsamp', 'gravSensors', 'gravReadings']
#data_names_list_sensorplots = ['magSensors_rndsub', 'magReadings_rndsub', 'gravSensors_rndsub', 'gravReadings_rndsub']

fpath_samples_list = [
    os.path.join(dir_parent, xp(exp_str, num), fname_samples)
    for num in num_list
]

fpath_csv_list = [
    [
        os.path.join(dir_parent, xp(exp_str, num), "{}".format(data_name) + ".csv")
        for data_name in data_names_list_sensorplots
    ]
    for num in num_list
]

fieldobs_names_list = ['fieldobsSensors', 'fieldobsReadings']
fpath_fieldobs_csv_list = [
    [
        os.path.join(dir_parent, xp(exp_str, num), "{}".format(data_name) + ".csv")
        for data_name in fieldobs_names_list
    ]
    for num in num_list
]

dir_out_list = [
    os.path.join(dir_parent, xp(exp_str, num), dir_output_sensors)
    for num in num_list
]
print(dir_out_list)

In [ ]:
_ = plot_sensor_output(fpath_samples_list, dir_out_list, fpath_csv_list, data_names_list)

In [ ]:
sensors, readings = plot_fieldobs(
    fpath_samples_list, dir_out_list, fpath_fieldobs_csv_list, fieldobs_names_list
)

In [ ]:
print(predicted.shape)
print(actual.shape)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.DataFrame([actual, predicted], index=['actual', 'predicted']).T
not_missing_idx = df['actual'] != -1
df = df.loc[not_missing_idx, :].reset_index(drop = True)

In [ ]:
labels = [0, 1]
cnf = confusion_matrix(df['actual'], df['predicted'])
df_cnf = pd.DataFrame(cnf, columns=labels, index=labels)
df_cnf.index.name = 'True label'
df_cnf.columns.name = 'Predicted label'
df_cnf

In [ ]:
tp = df_cnf.loc[1,1]
tn = df_cnf.loc[0,0]
fn = df_cnf.loc[0,1]
fp = df_cnf.loc[1,0]
sensitivity = tn/(tn + fp)
specificity = tp/(tp + fn)
precision = tp/(tp + fp)
print(sensitivity, specificity, precision)


In [ ]:
import sklearn.metrics
# true positive / total positives
print(sklearn.metrics.precision_score(df['actual'], df['predicted']))
# true positive / (true pos + false neg)
print(sklearn.metrics.recall_score(df['actual'], df['predicted']))

In [ ]:
df_cnf.to_csv('/Users/davidkohn/Desktop/confusion_matrix.csv')

In [ ]:
df_cnf

# Table generation

In [ ]:
fpath = fpath_samples_list[0]
samples = np.load(fpath)

In [ ]:
fr = samples['fieldReadings']

In [ ]:
n = fr.shape[0]
fr_uncertainty = fr.sum(axis = 0)/n

In [ ]:
fr_uncertainty.max()

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.gca()
color_data = fr_uncertainty * 100
color_data = np.round(fr_uncertainty,2) * 100
plt.scatter(
    sensors['x'], sensors['y'],
    c = color_data
)
_ = plt.xlabel('Eastings (km)')
_ = plt.ylabel('Northings (km)')
ytl = ax.get_yticklabels()
cbt = [0, 50, 99]
cbtl = ['{}\%'.format(cbt_i) for cbt_i in cbt]
cb = plt.colorbar(ticks = cbt)
cb.set_ticklabels(cbtl)
cb.set_label('\% of samples equal to 1')

yt = ax.get_yticks()[1:-1]
ytl = ['{}'.format(yt_i/1000) for yt_i in yt]
ax.set_yticks(yt)
ax.set_yticklabels(ytl)
xt = ax.get_xticks()[1:-1]
xtl = ['{}'.format(xt_i/1000) for xt_i in xt]
ax.set_xticks(xt)
ax.set_xticklabels(xtl)
plt.savefig('/Users/davidkohn/Desktop/fieldobs-uncertainty.eps')

In [ ]:

cc_og = pd.Series(color_data)
other_bool = ~cc_og.isin([0,1,100])
cc = cc_og[other_bool]
cc_vc = cc.value_counts(ascending = False)
gt10_bool = (cc_vc.index > 10) & (cc_vc.index < 90)
cc_vc[gt10_bool].shape
#cc.value_counts()

In [ ]:
color_data = fr_uncertainty * 100
color_data = np.round(fr_uncertainty,2) * 100
cc_og = pd.Series(color_data)
cc_bool = (cc_og < 90) & (cc_og > 10)
cc_new = cc_og[cc_bool]
sensors_cc = sensors.loc[cc_bool, :]

fig = plt.figure(figsize=(5,5))
ax = plt.gca()
color_data = fr_uncertainty * 100
color_data = np.round(fr_uncertainty,2) * 100
plt.scatter(
    sensors_cc['x'], sensors_cc['y'],
    c = cc_new
)
_ = plt.xlabel('Eastings (km)')
_ = plt.ylabel('Northings (km)')
ytl = ax.get_yticklabels()
cbt = [cc_new.min(), (cc_new.max() + cc_new.min())/2, cc_new.max()]
cbtl = ['{:.0f}\%'.format(cbt_i) for cbt_i in cbt]
cb = plt.colorbar(ticks = cbt)
cb.set_ticklabels(cbtl)
cb.set_label('\% of samples equal to 1')

yt = ax.get_yticks()[1:-1]
ytl = ['{}'.format(yt_i/1000) for yt_i in yt]
ax.set_yticks(yt)
ax.set_yticklabels(ytl)
xt = ax.get_xticks()[1:-1]
xtl = ['{}'.format(xt_i/1000) for xt_i in xt]
ax.set_xticks(xt)
ax.set_xticklabels(xtl)
plt.savefig('/Users/davidkohn/Desktop/fieldobs-uncertainty-sub.eps')

In [ ]:
cc_new

In [ ]:
[ytl_i.get_text() for ytl_i in ytl]

In [ ]:
f = plt.figure(figsize=(5,5))
ax = plt.gca()
n_samples = fr.shape[0]
n_points = fr.shape[1]
fr_sum = fr.sum(axis = 1)
fracs = fr_sum / n_points
weights = np.ones((n_samples,1))/n_samples
plt.hist(
    fracs, bins = 20, weights = weights, fill=None
)

mean = np.mean(fracs)
plt.axvline(mean,c='red')

yt = ax.get_yticks()[1:-1]
ytl = ['{:.0f}\%'.format(yt_i * 100) for yt_i in yt]
_ = ax.set_yticks(yt)
_ = ax.set_yticklabels(ytl)

xt = ax.get_xticks()[1:-1]
xtl = ['{:.0f}\%'.format(xt_i * 100) for xt_i in xt]
_ = ax.set_xticks(xt)
_ = ax.set_xticklabels(xtl)

_ = plt.xlabel('$\hat{p}$')
_ = plt.ylabel('Posterior probability')
plt.savefig('/Users/davidkohn/Desktop/fieldobs-p.eps')

In [ ]:
np.mean(fracs)

In [ ]:
layer_title_list = ['Layer 0', 'Layer 1']
p_list = [p.capitalize() for p in param_title_list]

table = pd.DataFrame(0, index = layer_title_list, columns = p_list)
print(table)

In [ ]:
for samples_key, layer_title in zip(param_key_list, layer_title_list):
    for param_idx, param_title in zip(param_idx_list, p_list):
        param = samples[samples_key][:, param_idx]
        stat = param.mean()
        #print('{} {} {}'.format(samples_key, param_title, param.mean()))
        table.loc[layer_title, param_title] = stat

In [ ]:
table

In [ ]:
table.to_csv('/Users/davidkohn/Desktop/mean.csv')

# Testing

In [ ]:
## Testing making trace plots for each chain

In [ ]:
#fpath = '/Volumes/david_hd/obsidian/output/experiments/gascoyne_v5_run03/gascoyne_v5-rs-run03-thin10000.npz'
#fpath = '/Volumes/david_hd/obsidian/output/experiments/06_22_2018/07/output2.npz'

fpath = fpath_samples_list[0]
samples = np.load(fpath)
print(samples.keys())

In [ ]:
stack_length_list = [1160161, 1154009, 1129264, 1111255]

In [ ]:
thinned_lengths = [(l - 1000)/1000 for l in stack_length_list]
print(thinned_lengths)
thinned_lengths2 = [np.floor(l) for l in thinned_lengths]
print(thinned_lengths2)

In [ ]:
cum_lengths = np.cumsum(thinned_lengths2)
start = 0
for end in cum_lengths:
    print(start, end)
    data = param[start:end]
    plt.plot(data)
    start = end

In [ ]:
cum_lengths

In [ ]:
fr = samples['fieldReadings']
np.unique(fr)

In [ ]:
param = samples['layer0rockProperties'][:,0]

In [ ]:
param.shape

In [ ]:
samples['layer0rockProperties'].shape

In [ ]:
samples['gravReadings']

In [ ]:
pymc3.diagnostics.gelman_rubin(param)

In [ ]:
pymc3.diagnostics.effective_n(param)

In [ ]:
geweke = pymc3.diagnostics.geweke(grav)

In [ ]:
pymc3.plots.autocorrplot(param)

In [ ]:
grav.shape

In [ ]:
pymc3.plots.traceplot(grav)

In [ ]:
pymc3.diagnostics.autocov(grav)

In [ ]:
data = np.random.randn(50)
f = plt.figure()
ax = plt.gca()
plt.plot(data)
yt = ax.get_yticks()
print(yt)
ytl = ['{:.0f}\%'.format(yt_i * 100) for yt_i in yt]
_ = ax.set_yticklabels(ytl)
#xtl

In [ ]:
data = np.random.randn(100)
h = plt.hist(data, bins = 10)

In [ ]:
f = plt.figure(figsize = (10,5))
ax = plt.gca()
labels = pd.rolling_mean(h[1], 2)[1:]
label_strs = ['{:.1f}'.format(l) for l in labels]
counts = h[0]
plt.bar(
    labels, counts, align='center',
    edgecolor='black', width = 0.5, fill = False
)
_ = ax.set_xticks(labels)
_ = ax.set_xticklabels(label_strs)

In [ ]:
#data.mean()
np.argmin([np.abs(l - data.mean())  for l in labels])

In [ ]:
plt.bar(labels,counts, align = 'center', width = 0.5)

In [ ]:
np.any(None)

In [ ]:
data = np.load(fpath_samples_list[0])
mag = data['magReadings']
data = mag.mean(axis = 0) - mag.mean()

In [ ]:
bins = 10
weights = np.ones_like(data)/float(len(data))
h = plt.hist(
    data, 
    bins = bins, weights = weights,
)

In [ ]:
plt.clf()
labels = pd.rolling_mean(h[1], 2)[1:]
label_strs = ['{:.1f}'.format(l) for l in labels]
counts = h[0]

fig = plt.figure(figsize = (5, 5))
ax = plt.gca()
#width = 0.5
width = 1.0
plt.bar(
    labels, counts, width = 100
    #align='center',
    #edgecolor='black', 
    #width = width, 
    #fill = False
)
#_ = ax.set_xticks(labels)
#_ = ax.set_xticklabels(label_strs)

In [ ]:
labels.shape

In [ ]:
counts.shape

# Voxel plots

In [ ]:
# put together file paths
dir_output_voxels = 'output/plots-voxels'
search_str = 'voxel'
#dir_voxels = 'voxels'
dir_voxels = ''
if author == 'dk':
    fpath_in_list = [
        get_fname_list(os.path.join(dir_parent, exp_str + num, dir_voxels), search_str)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str, num, dir_output_voxels)
        for num in num_list
    ]
elif author == 'rs':
    fpath_in_list = [
        get_fname_list(os.path.join(dir_parent, exp_str + num), search_str)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str + num, dir_output_voxels)
        for num in num_list
    ]
print(fpath_in_list)
print(dir_out_list)

In [ ]:
dir_out_list

In [ ]:
fname_save_template_str = 'posterior-layer-{:02}.{}'
view_list = []
for voxel_path_list, dir_out in zip(fpath_in_list, dir_out_list):
    if voxel_path_list:
        view = plot_layer_posteriors(voxel_path_list, dir_out)
        view_list.append(view)

In [ ]:
f = fpath_in_list[0][0]
v = np.load(f)

In [ ]:
v.keys()

In [ ]:
v['boundary0'].shape

In [ ]:
fb_layer0 = view.fbounds[0]
fb_layer1 = view.fbounds[1]
#fb = fb.mean(axis = 0)
#cond = fb == 0
#print(cond.sum())
#print(71 * 36 * 36)

In [ ]:
sample_idx = -1
fb_layer0_sample = fb_layer0[sample_idx,:,:]
fb_layer1_sample = fb_layer1[sample_idx,:,:]

In [ ]:
fb_layer0_sample.size

In [ ]:
zero_cond = fb_layer0_sample == 0 
zero_cond.sum()

In [ ]:
zero_cond = fb_layer1_sample == 0 
zero_cond.sum()

In [ ]:
x, y = np.where(cond)

In [ ]:
x[0], y[0]

In [ ]:
fb[x[0], y[0]]

In [ ]:
view_idx = 0
layer_idx = 1
sample_idx = -1

view = view_list[view_idx]
layer_voxels = view.layers[layer_idx][0,:,:,:]

In [ ]:
view.layers[0].shape

In [ ]:
print(view.layers[0][-1, :, 0, 0].reshape(-1, 1))

In [ ]:
print(view.layers[1][-1, :, 0, 0].reshape(-1, 1))

In [ ]:
print(view.fbounds[0][-1,0,0])
print(view.fbounds[1][-1,0,0])

In [ ]:
layer_voxels[x[0], y[0], 0]

In [ ]:
view.rockprops['Density'].shape

In [ ]:
view.layers[0].shape

In [ ]:
layer_voxels[0,0,:]

In [ ]:
view_idx = 0
layer_idx = 0
sample_idx = -1

view = view_list[view_idx]
layer_voxels = view.layers[layer_idx]
sample_voxels = layer_voxels[sample_idx,:,:,:]

fb = view.fbounds[layer_idx]

In [ ]:
mean_layer_bounds = fb.mean(axis = 0)

In [ ]:
mean_layer_bounds.sum()

In [ ]:
(view.layers[0][-1,:,:,:] + view.layers[1][-1,:,:,:]).sum()

In [ ]:
(view.layers[0][-1,:,:,0] != 0).sum()

In [ ]:
344/(36*82)

In [ ]:
l.shape

In [ ]:
l = view.layers[0][-1,:,:,:]
l[0, :, :]

In [ ]:
l = view.layers[1][-1,:,:,:]
l[0, :, :]

In [ ]:
view.fbounds[0][-1]

In [ ]:
voxel_size = view.zbounds[1]/12

In [ ]:
view.fbounds[1][-1].flatten().shape

In [ ]:
(view.fbounds[0][-1, :, :] == 0).sum()

In [ ]:
(view.fbounds[1][-1, :, :] == 0).sum()

In [ ]:
(view.fbounds[0][-1].flatten() == 0).sum()

In [ ]:
(view.fbounds[1][-1].flatten() == 0).sum()

In [ ]:
view.fbounds[1][-1].flatten().shape

In [ ]:
(view.fbounds[1][-1].flatten() < voxel_size).sum()

In [ ]:
# view = a specific experiment in the list
view_idx = 0
layer_idx = 0

view = view_list[view_idx]
layer_voxels = view.meanlayer(layer_idx)
#view_idx = 0
#layer_idx = 1
#sample_idx = -1

#view = view_list[view_idx]
#layer_voxels = view.layers[layer_idx][sample_idx,:,:,:]
#l = view.layers[0][-1,:,:,:]

app = vv.use()
vv.figure(1)
vv.xlabel('Eastings (units)')
vv.ylabel('Northings (units)')
vv.zlabel('Depth (units)')
a = vv.gca()
print(a.GetView())
#a.loc = (0, 0, 100)
#a.azimuth = 100
#a.camera.fov = 10
a.daspect = 1, 1, -1
a.camera.elevation = -90
#a.camera.azimuth = 30
a.camera.roll = 10
print(a.GetView())
print(a.camera)
#a.camera.
a.SetView()
#t = vv.volshow(l, cm=vv.CM_JET)
#t = vv.volshow(layer_voxels, cm=vv.CM_JET, renderStyle='ray')
t = vv.volshow(layer_voxels, cm=vv.CM_JET)
vv.ColormapEditor(a)
app.Run()

In [ ]:
view_list

In [ ]:
# manually run visvis so you can move the plot around
fpath_idx = 6
layer_idx = 0
fn = fpath_in_list[fpath_idx][0]
print('filename: {}'.format(fn)
n_layers = len(view.layers)
print('no layers: {}'.format(n_layers)
view = geovis_notebook_version.MasonView(fn)
layer_voxels = view.meanlayer(layer_idx)

app = vv.use()
vv.figure(1)
vv.xlabel('Eastings (units)')
vv.ylabel('Northings (units)')
vv.zlabel('Depth (units)')
a = vv.gca()
print(a.GetView())
#a.loc = (0, 0, 100)
#a.azimuth = 100
#a.camera.fov = 10
a.daspect = 1, 1, -1
a.camera.elevation = -90
#a.camera.azimuth = 30
a.camera.roll = 10
print(a.GetView())
print(a.camera)
#a.camera.
a.SetView()
t = vv.volshow(layer_voxels, cm=vv.CM_JET)
#t = vv.volshow(layer_voxels, cm=vv.CM_JET, renderStyle='ray')
vv.ColormapEditor(a)
app.Run()